# Stack

The **Stack** class provides the same features as **ROOT**'s **THStack** class but with extended functionality and improved ease of use. So make sure to also consult the [ROOT documentation](https://root.cern.ch/doc/master/classTHStack.html).

Again we start by creating a test sample that we can play with:

In [ ]:
import ROOT
from mephisto import IOManager, Histo1D, Stack

IOManager.CreateTestSample("testsample.root", tree="TestTree", nbranches=10, nevents=1e4)

Now let's define some histograms that we can put into our Stack. The most likely for Stacks use cases are probably data vs. background comparison and background vs. signal comparison plots.

One way we could start is by defining the properties of these histograms and how to fill them with out dummy data. If you already took the IOManager and Histo1D tutorial the following will already look familiar to you:

In [ ]:
histoconfig = {
    # background histograms:
    "bkg1": {
        "fill":  dict(varexp="branch_1", weight="0.6*branch_10"),
        "style": dict(template="background", fillcolor="#6baff4", title="ttbar")
    },
    "bkg2": {
        "fill":  dict(varexp="branch_2", weight="0.5*branch_10"),
        "style": dict(template="background", fillcolor="#fff266", title="W+jets")
    },
    "bkg3": {
        "fill":  dict(varexp="branch_3", weight="0.4*branch_10"),
        "style": dict(template="background", fillcolor="#98ff79", title="Single top")
    },
    "bkg4": {
        "fill":  dict(varexp="branch_4", weight="0.3*branch_10"),
        "style": dict(template="background", fillcolor="#D182FF", title="Diboson")
    },
    "bkg5": {
        "fill":  dict(varexp="branch_5", weight="0.2*branch_10"),
        "style": dict(template="background", fillcolor="#ffc25e", title="Z+jets")
    },
    # signal histograms:
    "sig1": {
        "fill":  dict(varexp="branch_6", weight="0.5*branch_10"),
        "style": dict(template="signal", linecolor=ROOT.kRed, title="Signal 1")
    },
    "sig2": {
        "fill":  dict(varexp="branch_7", weight="0.4*branch_10"),
        "style": dict(template="signal", linecolor=ROOT.kBlue, title="Signal 2")
    },
    # data histogram:
    "data": {
        "style": dict(template="data", title="Data")
    }
}

Now let's create a few Histo1Ds and fill them with some dummy data:

In [ ]:
h = {}  # dictionary of histograms
binning = (25, 0.0, 10.0)

h["data"] = Histo1D("data", "", *binning)
for process, config in histoconfig.items():
    if process == "data":
        continue
    h[process] = Histo1D(process, "", *binning)
    h[process].Fill("testsample.root", tree="TestTree", **config["fill"])
    if process.startswith("bkg"):
        h["data"].Fill("testsample.root", tree="TestTree", append=True, **config["fill"])
        
# right now the data histogram is identical to the sum of all backgrounds,
# so let's add some variations to it to make it look more 'real':
import random
for bn in range(1, h["data"].GetNbinsX() + 1):
    h["data"].SetBinContent(bn, random.uniform(0.8, 1.2) * h["data"].GetBinContent(bn))
    h["data"].SetBinError(bn, random.uniform(1, 5) * h["data"].GetBinError(bn))

Next we create a Stack, register our histograms and draw it!

In [ ]:
s = Stack()
for process, histo in h.items():
    stackme = process.startswith("bkg")  # only stack background histograms on top one another
    s.Register(histo, stack=stackme, **histoconfig[process]["style"])
s.Print("mystack.png", ymin=100, xtitle="p_{T}(#mu)", xunits="GeV")

# view in in Jupyter:
from IPython.display import Image
Image(filename='mystack.png')

As with Histo1Ds, you can change any Stack attribute by adding the property and desired value as a keyword argument to the constructor or the ```Print``` method.

If you don't know what is meant by properties, you'd probably didn't do the Histo1D tutorial. Here's the short answer: Any attribute of a class for which there is a setter method is a property. For Stack objects you can also change the properties of the *'sum-of-stacked-histograms'*-histogram (by default called 'Total SM' in the legend above) by adding the prefix ```stacksum``` in front of the Histo1D property. For example:

In [ ]:
s.Print("mystack.png", ymin=100, xtitle="p_{T}(#mu)", xunits="GeV", stacksumlinecolor=ROOT.kRed)

Image(filename='mystack.png')

As with any Histo1D object you can also change the legend and errorband properties with the respective prefixes, e.g. for the total-stack-histogram this would then add up to ```stacksumerrorbandfillstyle``` in order to change the fill style of the total-stack's errorband.

Often one would like to view the data vs. MC ratio, background purity or signal significance in the same final plot. You can add one or more of them as additional pads below the main Stack by setting the flags ```ratio```, ```contribution```and/or ```sensitivity``` to **True**, respectively:

In [ ]:
s.Print(
    "mystack.png",
    ymin=100,
    xtitle="p_{T}(#mu)",
    xunits="GeV",
    ratio=True,
    contribution=True,
    sensitivity=True)

Image(filename='mystack.png')

You can print the yields to ```stdout``` and also as a ```.csv```, ```.tex``` or ```.pdf``` file:

In [ ]:
s.PrintYieldTable("table.pdf", precision=1)

from IPython.display import IFrame, display
display(IFrame("table.pdf", width=500, height=350))  # you might need to use Firefox for this...

So much for the basics. You're good to go now. Cheers!